# Processing raw XMM-Newton Pointed data

This tutorial will teach you how to use DAXA to process raw XMM-Newton data into a science ready state using one line of Python code (or several lines, if you wish to have more control over the settings for each step). **This relies on there being an initialised (either manually before launching Python, or in your bash profile/rc) backend installation of the XMM Science Analysis System (SAS), including accessible calibration files** - DAXA will check for such an installation, and will not allow processing to start without it.

**All DAXA processing steps will parallelise as much as possible - processes running on different ObsIDs/instruments/sub-exposures will be run simultaneously (if cores are available)**

## Import Statements

In [23]:
from daxa.mission import XMMPointed
from daxa.archive import Archive
from daxa.process.simple import full_process_xmm
from daxa.process.xmm.setup import cif_build, odf_ingest
from daxa.process.xmm.assemble import emchain, epchain, cleaned_evt_lists, merge_subexposures
from daxa.process.xmm.check import emanom
from daxa.process.xmm.clean import espfilt

## An Archive to be processed

Every processing function implemented in DAXA takes an Archive instance as its first argument; if you don't already know what that is then you should go back and read the following tutorials:

* [Creating a DAXA archive](archives.html) - This explains how to create an archive, load an existing archive, and the various properties and features of DAXA archives.
* [Using DAXA missions](missions.html) - Here we explain what DAXA mission classes are and how to use them to select only the data you need.

Here we create an archive of XMM observations of the galaxy cluster Abell 907:

In [5]:
xm = XMMPointed()
xm.filter_on_name('A907')
arch = Archive('Abell907', xm)

/Users/dt237/code/DAXA/daxa/mission/xmm.py:83: UserWarning: 140 of the 17689 observations located for this mission have been removed due to NaN RA or Dec values
  self._fetch_obs_info()


## One-line solution

Though we provide individual functions that wrap the various steps required to reduce and prepare XMM data, and they can be used separately for greater control over the configuration parameters, we also include a one-line solution which executes the processing steps with default configuration.

We believe that the default parameters are adequate for most use cases, and this allows for users unfamiliar with the intricacies of XMM data to easily start working with it. Executing the following will automatically generate cleaned event lists for MOS1, MOS2, PN, RGS1, and RGS2 (if they were selected during mission creation), as well as images and exposure maps for MOS1, MOS2, and PN:

In [6]:
full_process_xmm(arch)

XMM-Newton Pointed - Generating calibration files: 100%|████████████████████████| 3/3 [00:19<00:00,  6.51s/it]
XMM-Newton Pointed - Generating ODF summary files: 100%|████████████████████████| 3/3 [00:03<00:00,  1.25s/it]
XMM-Newton Pointed - Assembling PN and PN-OOT event lists: 100%|████████████████| 3/3 [03:37<00:00, 72.36s/it]
XMM-Newton Pointed - Assembling MOS event lists: 100%|██████████████████████████| 6/6 [01:22<00:00, 13.70s/it]
XMM-Newton Pointed - Finding PN/MOS soft-proton flares: 100%|███████████████████| 9/9 [00:17<00:00,  1.93s/it]
XMM-Newton Pointed - Generating cleaned PN/MOS event lists: 100%|███████████████| 8/8 [00:03<00:00,  2.62it/s]
XMM-Newton Pointed - Generating final PN/MOS event lists: 100%|████████████████| 8/8 [00:00<00:00, 120.67it/s]
Generating products of type(s) expmap: 100%|████████████████████████████████████| 8/8 [00:22<00:00,  2.84s/it]


## General arguments for all processing functions

There are some arguments that all processing functions will take - they don't control the behaviour of the processing step itself, but instead how the commands are executed:

* **num_cores** - The number of cores that the function is allowed to use. This is set globally by the daxa.NUM_CORES constant (if set before any other DAXA function is imported), and by default is 90% of the cores available on the system.
* **timeout** - This controls how long an individual instance of the process is allowed to run before it is cancelled; the whole function may run for longer depending how many pieces of data need processing and how many cores are allocated. The default is generally null, but it can be set using an astropy quantity with time units.

## Breaking down the XMM processing steps

### Setup steps

These functions set up the necessary environment/files for the further processing/reduction of XMM data.

#### Building calibration files (cif_build)

The [cif_build function](../../daxa.process.xmm.html#daxa.process.xmm.setup.cif_build) function is a wrapper for the XMM SAS tool of the same name - it assembles a CIF, which points other SAS tools to the current calibration files required for whatever XMM instrument they are working on.

Only the `analysis_date` parameter of this function affects the files produced by this function (other things can be controlled, such as the number of cores or timeout) - this date determines which calibration files are selected for the generated CIF, and the default is the current date - another date can be passed as a Python datetime object.

**All other processing steps depend on this one**

In [12]:
help(cif_build)

Help on function cif_build in module daxa.process.xmm.setup:

cif_build(obs_archive: daxa.archive.base.Archive, num_cores: int = 9, disable_progress: bool = False, analysis_date: Union[str, datetime.datetime] = 'now', timeout: astropy.units.quantity.Quantity = None) -> Tuple[dict, dict, dict, str, int, bool, astropy.units.quantity.Quantity]
    A DAXA Python interface for the SAS cifbuild command, used to generate calibration files for XMM observations
    prior to processing. The observation date is supplied by the XMM mission instance(s), and is the date when the
    observation was started (as acquired from the XSA).
    
    :param Archive obs_archive: An Archive instance containing XMM mission instances for which observation calibration
        files should be generated. This function will fail if no XMM missions are present in the archive.
    :param int num_cores: The number of cores to use, default is set to 90% of available.
    :param bool disable_progress: Setting this to tr

#### Summarising the available data files (odf_ingest)

The [odf_ingest function](../../daxa.process.xmm.html#daxa.process.xmm.setup.odf_ingest) function simply examines the observation data file (ODF) directory, and determines the instruments (including observing modes and sub-exposures) that have data present - it then creates a SAS summary file.

Our implementation of this function wraps the original SAS tool, and then adds a second step - this parses the SAS summary file and extracts the information that is relevant to whether we can use a particular instrument (and sub-exposure of an instrument) for astrophysics. This is what populates the XMM entry in the `observation_summaries` property of the archive class.

**All subsequent steps depend on this one - and observation_summaries will have no XMM entry until this is run**

In [13]:
help(odf_ingest)

Help on function odf_ingest in module daxa.process.xmm.setup:

odf_ingest(obs_archive: daxa.archive.base.Archive, num_cores: int = 9, disable_progress: bool = False, timeout: astropy.units.quantity.Quantity = None)
    This function runs the SAS odfingest task, which creates a summary of the raw data available in the ODF
    directory, and is used by many SAS processing tasks.
    
    :param Archive obs_archive: An Archive instance containing XMM mission instances for which observation summary
        files should be generated. This function will fail if no XMM missions are present in the archive.
    :param int num_cores: The number of cores to use, default is set to 90% of available.
    :param bool disable_progress: Setting this to true will turn off the SAS generation progress bar.
    :param Quantity timeout: The amount of time each individual process is allowed to run for, the default is None.
        Please note that this is not a timeout for the entire odf_ingest process, but 

### EPIC Cameras (CCD)

#### Assembling whole-camera MOS initial event lists (emchain)

The [emchain function](../../daxa.process.xmm.html#daxa.process.xmm.assemble.emchain) is what assembles the raw, separate-CCD-level, event lists and files into a single raw events list for a whole XMM MOS camera exposure. If there are **multiple sub-exposures** which DAXA has identified as being usable, then a separate raw event list is created for each of them - they are combined in a later processing step.

We have implemented this method so that MOS1 and MOS2 (and sub-exposures, if present) data are processed in parallel for a given observation (and all observations are processed in parallel as well).

Only the `process_unscheduled` argument passed to emchain can change the files produced, as it controls whether unscheduled sub-exposures should be processed or not - the default is True.

In [15]:
help(emchain)

Help on function emchain in module daxa.process.xmm.assemble:

emchain(obs_archive: daxa.archive.base.Archive, process_unscheduled: bool = True, num_cores: int = 9, disable_progress: bool = False, timeout: astropy.units.quantity.Quantity = None)
    This function runs the emchain SAS process on XMM missions in the passed archive, which assembles the
    MOS-specific ODFs into combined photon event lists - rather than the per CCD files that existed before. The
    emchain manual can be found here (https://xmm-tools.cosmos.esa.int/external/sas/current/doc/emchain.pdf) and
    gives detailed explanations of the process.
    
    The DAXA wrapper does not allow emchain to automatically loop through all the sub-exposures for a given
    ObsID-MOSX combo, but rather creates a separate process call for each of them. This allows for greater
    parallelisation (if on a system with a significant core count), but also allows the same level of granularity
    in the logging of processing of diffe

#### Assembling whole-camera PN initial event lists (epchain)

The [epchain function](../../daxa.process.xmm.html#daxa.process.xmm.assemble.epchain) serves much the same purpose as emchain, but for PN camera data. There is only one PN camera, but the presence of multiple sub-exposures is still possible. This function generates raw, whole-camera, event lists just like emchain, but also creates whole-camera out-of-time (OOT) event lists, which attempts to identify event lists that were detected during a CCD readout (this can leave a characteristic streak in the direction of readout for bright sources in PN observations).

In [16]:
help(epchain)

Help on function epchain in module daxa.process.xmm.assemble:

epchain(obs_archive: daxa.archive.base.Archive, process_unscheduled: bool = True, num_cores: int = 9, disable_progress: bool = False, timeout: astropy.units.quantity.Quantity = None)
    This function runs the epchain SAS process on XMM missions in the passed archive, which assembles the
    PN-specific ODFs into combined photon event lists - rather than the per CCD files that existed before. A run of
    epchain for out of time (OOT) events is also performed as part of this function call. The epchain manual can be
    found here (https://xmm-tools.cosmos.esa.int/external/sas/current/doc/epchain.pdf) and gives detailed
    explanations of the process.
    
    Per the advice of the SAS epchain manual, the OOT event list epchain call is performed first, and its intermediate
    files are saved and then used for the normal call to epchain.
    
    :param Archive obs_archive: An Archive instance containing XMM mission instanc

#### Searching for anomalous MOS CCD states (emanom) 

The [emanom function](../../daxa.process.xmm.html#daxa.process.xmm.check.emanom) wraps the SAS function of the same name, and is an **optional** step that finds MOS CCDs which have operated in an 'anomalous' state, where the  background at E < 1 keV is strongly enhanced. However, it should be noted that the "anonymous" anomalous state of MOS1 CCD#4 is not always detectable from the unexposed corner data.

**We note that this is not enabled by default in the `full_process_xmm` function, and your results may vary**.

In [26]:
help(emanom)

Help on function emanom in module daxa.process.xmm.check:

emanom(obs_archive: daxa.archive.base.Archive, num_cores: int = 9, disable_progress: bool = False, timeout: astropy.units.quantity.Quantity = None)
    This function runs the SAS emanom function, which attempts to identify when MOS CCDs are have operated in an
    'anomalous' state, where the  background at E < 1 keV is strongly enhanced. Data above 2 keV are unaffected, so
    CCDs in anomalous states used for science where the soft X-rays are unnecessary do not need to be excluded.
    
    The emanom task calculates the (2.5-5.0 keV)/(0.4-0.8 keV) hardness ratio from the corner data to determine
    whether a chip is in an anomalous state. However, it should be noted that the "anonymous" anomalous state of
    MOS1 CCD#4 is not always detectable from the unexposed corner data.
    
    This functionality is only usable if you have SAS v19.0.0 or higher - a version check will be performed and
    a warning raised (though no e

#### Searching for flared periods of the observations (espfilt)

This particularly important function ([espfilt](../../daxa.process.xmm.html#daxa.process.xmm.clean.espfilt)) wraps the SAS tool of the same name, and attempts to automatically detect periods of observations which have been overwhelmed by soft-proton flaring. The ultimate result of this process is a set of 'good time intervals', which are periods of a particular observation that have been deemed safe to use; **this check if performed on all instruments and sub-exposures of an observation independently**.

Several configuration options are provided, mirroring those that can be passed into the espfilt SAS tool:

* **method** - There are two methods of identifying flaring available, 'histogram' (the default) and 'ratio'. The histogram method fits a gaussian to the whole-FoV count-rate distribution (in order to find a mean and a standard deviation), and then defines good-time intervals by finding the time periods where the whole-FoV lightcurve count-rate is within `allowed_sigma` of the mean. The 'ratio' method compares the count-rate from the FoV with the count-rate in the unexposed corners of XMM observations (i.e. there is an active CCD but the telescope is not focusing photons or soft-protons onto it); an acceptable ratio is defined by `ratio`, and points where the lightcurve ratio is lower than or equal to this are considered good-time intervals.
* **with_smoothing** - Should smoothing be applied to the light curve data. The default is True, in which case a smoothing factor of 51 seconds is used - an astropy quantity with time units may also be passed to set a custom value.
* **with_binning** - Should binning be applied to the light curve data. The default is True, in which case a binning size of 60 seconds is used - an astropy quantity with time units may also be passed to set a custom value.
* **ratio** - The acceptable ratio for the 'ratio' method.
* **filter_lo_en** - The lower energy bound used for soft-proton flaring identification; the default is 2.5 keV.
* **filter_hi_en** - The upper energy bound used for soft-proton flaring identification; the default is 8.5 keV.
* **range_scale** - Histogram fit range scale factor. The default is a dictionary with an entry for 'pn' (15.0) and an entry for 'mos' (6.0).
* **allowed_sigma** - Number of stdev from the mean count-rate allowed before a period is considered to **not** be a good time interval. Default is 3.
* **gauss_fit_lims** - The parameter limts for gaussian fits for the histogram method, the default is (0.1, 6.5).

In [20]:
help(espfilt)

Help on function espfilt in module daxa.process.xmm.clean:

espfilt(obs_archive: daxa.archive.base.Archive, method: str = 'histogram', with_smoothing: Union[bool, astropy.units.quantity.Quantity] = True, with_binning: Union[bool, astropy.units.quantity.Quantity] = True, ratio: float = 1.2, filter_lo_en: astropy.units.quantity.Quantity = <Quantity 2500. eV>, filter_hi_en: astropy.units.quantity.Quantity = <Quantity 8500. eV>, range_scale: dict = None, allowed_sigma: float = 3.0, gauss_fit_lims: Tuple[float, float] = (0.1, 6.5), num_cores: int = 9, disable_progress: bool = False, timeout: astropy.units.quantity.Quantity = None)
    The DAXA wrapper for the XMM SAS task espfilt, which attempts to identify good time intervals with minimal
    soft-proton flaring for individual sub-exposures (if multiple have been taken) of XMM ObsID-Instrument
    combinations. Both EPIC-PN and EPIC-MOS observations will be processed by this function.
    
    This function does not generate final event li

#### Applying event filtering and good-time-intervals to the raw event lists (cleaned_evt_lists)

The [cleaned_evt_lists](../../daxa.process.xmm.html#daxa.process.xmm.assemble.cleaned_evt_lists) function uses SAS' evselect tool to both apply the GTIs generated by the espfilt function, and to apply further filtering of events as directed by the user. 

We allow for filtering based on flag, pattern, energy, and MOS anomolous states (if the emanon task was run) - the user can also define custom filtering expressions. The MOS and PN filtering statements are defined separately, as it is standard practise to use different filtering for the different instruments. The exact behaviour is controlled by passing the following arguments:

* **lo_en** - The lowest energy of event that should be left in the output cleaned event list. The default is None, in which case no energy filter is applied.
* **hi_en** - The highest energy of event that should be left in the output cleaned event list. The default is None, in which case no energy filter is applied.
* **pn_filt_expr** - This essentially lets the user set the evselect expression keyword, and determines what filters should be applied to the PN event lists (other than energy range and good time interval application) - the default is ("#XMMEA_EP", "(PATTERN <= 4)", "(FLAG .eq. 0)"), where the elements of the tuple are combined into the base filtering expression.
* **mos_filt_expr** - This essentially lets the user set the evselect expression keyword, and determines what filters should be applied to the MOS event lists (other than energy range and good time interval application) - the default is ("#XMMEA_EM", "(PATTERN <= 12)", "(FLAG .eq. 0)"), where the elements of the tuple are combined into the base filtering expression.
* **filt_mos_anom_state** - Whether the function should use the results of an 'emanom' run to identify and remove MOS CCDs that are in anomolous states. Default is 'False', meaning no such filtering would be applied.
* **acc_mos_anom_states** - A list/tuple of acceptable MOS CCD status codes found by emanom (status- G is good at all energies, I is intermediate for E<1 keV, B is bad for E<1 keV, O is off, chip not in use, U is undetermined (low band counts <= 0)).

In [22]:
help(cleaned_evt_lists)

Help on function cleaned_evt_lists in module daxa.process.xmm.assemble:

cleaned_evt_lists(obs_archive: daxa.archive.base.Archive, lo_en: astropy.units.quantity.Quantity = None, hi_en: astropy.units.quantity.Quantity = None, pn_filt_expr: Union[str, List[str]] = ('#XMMEA_EP', '(PATTERN <= 4)', '(FLAG .eq. 0)'), mos_filt_expr: Union[str, List[str]] = ('#XMMEA_EM', '(PATTERN <= 12)', '(FLAG .eq. 0)'), filt_mos_anom_state: bool = False, acc_mos_anom_states: Union[List[str], str] = ('G', 'I', 'U'), num_cores: int = 9, disable_progress: bool = False, timeout: astropy.units.quantity.Quantity = None)
    This function is used to apply the soft-proton filtering (along with any other filtering you may desire, including
    the setting of energy limits) to XMM-Newton event lists, resulting in the creation of sets of cleaned event lists
    which are ready to be analysed (or merged together, if there are multiple exposures for a particular
    observation-instrument combination).
    
    :param 

#### Finalising the event lists by combining sub-exposures (merge_subexposures)

The [merge_subexposures](../../daxa.process.xmm.html#daxa.process.xmm.assemble.merge_subexposures) function will automatically check for cases where a particular instrument of a particular ObsID has multiple sub-exposures, and then it will automatically combine them. There are no user-provided arguments which will affect the output of this function, only the standard arguments to set number of cores and timeout.

In [24]:
help(merge_subexposures)

Help on function merge_subexposures in module daxa.process.xmm.assemble:

merge_subexposures(obs_archive: daxa.archive.base.Archive, num_cores: int = 9, disable_progress: bool = False, timeout: astropy.units.quantity.Quantity = None)
    A function to identify cases where an instrument for a particular XMM observation has multiple
    sub-exposures, for which the event lists can be merged. This produces a final event list, which is a
    combination of the sub-exposures.
    
    For those observation-instrument combinations with only a single
    exposure, this function will rename the cleaned event list so that the naming convention is comparable
    to the merged event list naming convention (i.e. sub-exposure identifier will be removed).
    
    :param Archive obs_archive: An Archive instance containing XMM mission instances for which cleaned event lists
        should be created. This function will fail if no XMM missions are present in the archive.
    :param int num_cores: The 

### Reflection Grating Spectrometer (RGS)

## 